# Lab 11 - Association Rule Learning - Apriori Algorithm

During this lab we will explore association rule learning. It is a domain of data mining that
focuses on discovering interesting relationships between variables in transactional data. You will
familiarize yourself with basic concepts such as association rules, support, confidence, lift, and
leverage and you will implement the Apriori algorithm.

You can also deepen your understanding and knowledge by studying the relevant materials from
[Chapter 6 (pdf)](http://infolab.stanford.edu/~ullman/mmds/ch6.pdf) of "Mining of Massive Datasets"
\- http://www.mmds.org/.

## 1. The Dataset

We will use a dataset that represents transactions of customers from the Instacart online grocery
delivery platform. The "The Instacart Online Grocery Shopping Dataset 2017" was provided on the
Kaggle platform. Although it is not available at its original location, you can find the files at:
- https://www.kaggle.com/datasets/yasserh/instacart-online-grocery-basket-analysis-dataset
- or 
- https://www.kaggle.com/datasets/psparks/instacart-market-basket-analysis
- or 
- https://www.kaggle.com/datasets/suhasyogeshwara/instcart-market-analysis

It is a classic example of a market basket analysis dataset. Download the dataset, extract the files
and familiarize yourself with the data.

There are 6 files available:
- aisles.csv - aisles of the store
- departments.csv - departments of the store
- order_products__prior.csv - details of prior orders (historical data), tells which items
  (products) were bought together in one basket (transaction)
- order_products__train.csv - details of train orders (last order for each customer)
- orders.csv - orders of the customers
- products.csv - products of the store

In [ ]:
# write your code here


### 1.1 Products and Transactions

We are not going to use the whole information provided by the dataset. For now, we are interested in
the list of products that customers bought in each transaction. Let's focus on the
`order_products__prior.csv` (historical transactions data) and `products.csv` (for the names of the
products) files.

For example:

```python
>>> order_products_prior_df[order_products_prior_df["order_id"] == 2]

    order_id  product_id  add_to_cart_order  reordered
59         7       34050                  1          0
60         7       46802                  2          0
```

We can see that a customer bought both products (with id `34050` and `46802`) in one transaction. 

If we want to check the names of the products, we can use the `products.csv` file:

```python
>>> products_df[products_df["product_id"].isin([34050, 46802])]

       product_id      product_name  aisle_id  department_id
34049       34050      Orange Juice        31              7
46801       46802  Pineapple Chunks       116              1
```

---

Preprocess the dataset into a format that is appropriate for checking the co-occurrences of products
in transactions. It is up to you to decide what that format should be in order to perform the
computations efficiently. You will later need to answer questions such as "What is the overall
number of transactions?", "What is the number of transactions in which a specific product was
bought?", "What is the number of transactions in which two specific products were bought?", etc. You
may start with the original data layout and then return to this step if needed. You can work on
product IDs and use the `products.csv` file later to check the product names.

---

Find top 10 products that customers bought the most and least often.

---

<span style="color:gold">Note:</span> The dataset may suggest that the "in" relationship between
items and baskets corresponds to real-life "part of" relationship. This is true in this case:
products (items) are purchased together in larger transactions (baskets). However, you should be
aware that for association rule learning, the "in" relationship could be any arbitrary many-to-many
relationship, even if it appears "backwards" compared to real life. See [Section 6.1.2
(pdf)](http://infolab.stanford.edu/~ullman/mmds/ch6.pdf) for an example.


In [36]:
# write your code here


## 2 Association Rules

Association rules are rules that express the relationship between items in transactions. They are
usually presentented in the form `A -> B`, where `A` and `B` are subsets of items, and `A -> B`
means that if items from `A` are purchased, then items from `B` are also purchased. We will refer to
`A` as the antecedent and `B` as the consequent of the rule. It is quite common to consider
consequents as single items, as it is easier to interpret the rules. For example, if the client buys
bread and milk, then what other item is likely to be purchased with them?

Let's consider an association rule `A -> B`.There are several metrics that can be used to evaluate
its quality.  The most common metrics are:
- `support` - the ratio of transactions that contain items from both `A` and `B` to the total number
  of transactions (alternatively, just the number of such transactions); we can interpret it as how
  often the rule applies,
- `confidence` - the ratio of transactions that contain both `A` and `B` to the number of
  transactions that contain `A`; we can interpret it as the probability of purchasing `B` given that
  `A` was purchased,
- `coverage` - the ratio of transactions that contain items from `A` to the total number of
  transactions; we can interpret it as how often is `A` purchased; it informs us about the general
  popularity of items from `A` in transactions and what is the fraction of transactions that the
  rule is under consideration (no matter if `B` is purchased or not).
  
<span style="color:gold">Note:</span> The concept of coverage can be defined not only
for rules but also for itemsets. It is the ratio of transactions that contain the items from the
itemset to the total number of transactions.

---

Write functions that let you compute support, confidence, and coverage metrics for association
rules. It is up to you to define the expected association rule structure, e.g., a dictionary, a
tuple, a named tuple, a dataclass, etc.

Below, the product names are used for simplicity, but you should adapt this to the format you
decided on in the previous step. Use your functions to compute the support, confidence, and coverage
metrics of the following association rules:
- {Bread} -> {Milk}
- {Milk} -> {Bread}
- {Bread, Milk} -> {Butter}
- {Banana, Apple} -> {Milk}
- {Bread, Milk, Butter} -> {Eggs}

In [18]:
# write your code here

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

df = pd.read_csv("../data/lab-11/order_products__prior.csv")
products_df = pd.read_csv("../data/lab-11/products.csv")
aisles_df = pd.read_csv("../data/lab-11/aisles.csv")
departments_df = pd.read_csv("../data/lab-11/departments.csv")

# te = TransactionEncoder()
# te_ary = te.fit(q["product_id"]).transform(q["product_id"])
# w = pd.DataFrame(te_ary, columns=te.columns_)

q = df.groupby('order_id').agg({'product_id': lambda x: set(x)})
q

,product_id
order_id,
2,"{33120, 17794, 40141, 9327, 30035, 43668, 2898..."
3,"{17668, 24838, 17704, 46667, 21903, 17461, 326..."
4,"{26434, 32645, 10054, 21351, 22598, 39758, 348..."
5,"{48002, 45698, 18569, 37011, 15005, 8479, 9633..."
6,"{15873, 41897, 40462}"
...,...
3421079,{30136}
3421080,"{25122, 4932, 31717, 27845, 12935, 10667, 3806..."
3421081,"{38185, 32299, 3060, 35221, 12218, 20539, 12861}"


In [19]:
PRODUCT_COL = "product_id"

def cov_itemset(df, itemset: set[int], absolute=False, col=PRODUCT_COL):
    count = df[col].map(lambda basket: itemset.issubset(basket)).sum()
    return count if absolute else count / len(df)

def coverage(df, rule, absolute=False, col=PRODUCT_COL):
    antecedent, _ = rule
    return cov_itemset(df, antecedent, absolute, col)

def support(df, rule, absolute=False, col=PRODUCT_COL):
    antecedent, consequent = rule
    return cov_itemset(df, antecedent | consequent, absolute, col)

def confidence(df, rule, col=PRODUCT_COL):
    antecedent, consequent = rule
    return cov_itemset(df, antecedent | consequent, absolute=True, col=col) / cov_itemset(df, antecedent, absolute=True, col=col)

def show_product_name(products_df, product_id):
    return products_df[products_df["product_id"] == product_id]["product_name"].iloc[0]

print(coverage(q, ({25122}, {4932}), absolute=True))
print(support(q, ({25122}, {4932}), absolute=True))
print(confidence(q, ({25122}, {4932})))

print(show_product_name(products_df, 25122))
print(show_product_name(products_df, 4932))


1082
20
0.018484288354898338
Organic European Style Lightly Salted Butter
Vanilla Bean Ice Cream


In [4]:
from mlxtend.preprocessing import TransactionEncoder

# qq = q[:100].copy()
# te = TransactionEncoder()
# te_ary = te.fit_transform(qq["product_id"], sparse=False)
# w = pd.DataFrame(te_ary, columns=te.columns_)

te = TransactionEncoder()
te_ary = te.fit_transform(q["product_id"], sparse=True)
w = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=[str(i) for i in te.columns_])

w


/tmp/ipykernel_8593/3551946193.py:10: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  w = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=[str(i) for i in te.columns_])


,1,2,3,4,5,6,7,8,9,10,...,49679,49680,49681,49682,49683,49684,49685,49686,49687,49688
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3214869,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3214870,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3214871,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3214872,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
qq = df.groupby("product_id").size().sort_values(ascending=True)

In [21]:
qq[(qq > 10000)]

product_id
12845     10027
6948      10060
37524     10101
22959     10130
35914     10148
          ...  
47209    213584
21903    241921
21137    264683
13176    379450
24852    472565
Length: 492, dtype: int64

In [30]:
qq.name = "count"

In [38]:
qq.reset_index()

,product_id,count
0,20759,1
1,46741,1
2,5255,1
3,44495,1
4,11254,1
...,...,...
49672,47209,213584
49673,21903,241921
49674,21137,264683
49675,13176,379450


In [39]:
products_df

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [42]:
qqq = pd.merge(qq.reset_index(), products_df, how='inner', on='product_id')

In [43]:
qqq

,product_id,count,product_name,aisle_id,department_id
0,20759,1,7.04 Oz. Grahamfuls Banana Vanilla 8ct,61,19
1,46741,1,Seasoned Southern Style Red Beans And Rice,59,15
2,5255,1,Chardonnay Carneros,62,5
3,44495,1,Lowfat Cherry Lime Supernova Kefir Cultured Mi...,120,16
4,11254,1,Drink Distinct All Natural Soda Pineapple Coco...,100,21
...,...,...,...,...,...
49672,47209,213584,Organic Hass Avocado,24,4
49673,21903,241921,Organic Baby Spinach,123,4
49674,21137,264683,Organic Strawberries,24,4
49675,13176,379450,Bag of Organic Bananas,24,4


In [27]:
from mlxtend.frequent_patterns import apriori

res = apriori(w, min_support=0.003, max_len=3, low_memory=True, use_colnames=True)

In [30]:
res[res["itemsets"].map(len) > 2]

,support,itemsets
781,0.003411,"(21903, 21137, 13176)"
782,0.003603,"(13176, 21137, 27966)"
783,0.004686,"(13176, 47209, 21137)"
784,0.003794,"(21903, 47209, 13176)"
785,0.003549,"(13176, 47209, 27966)"
786,0.003350,"(21903, 24852, 47766)"


In [37]:
for itemsets in res[res["itemsets"].map(len) > 2]["itemsets"]:
    for item_id in itemsets:
        print(show_product_name(products_df, int(item_id)))

Organic Baby Spinach
Organic Strawberries
Bag of Organic Bananas
Bag of Organic Bananas
Organic Strawberries
Organic Raspberries
Bag of Organic Bananas
Organic Hass Avocado
Organic Strawberries
Organic Baby Spinach
Organic Hass Avocado
Bag of Organic Bananas
Bag of Organic Bananas
Organic Hass Avocado
Organic Raspberries
Organic Baby Spinach
Banana
Organic Avocado


In [45]:
qqq["product_name"]

0                   7.04 Oz. Grahamfuls Banana Vanilla 8ct
1               Seasoned Southern Style Red Beans And Rice
2                                      Chardonnay Carneros
3        Lowfat Cherry Lime Supernova Kefir Cultured Mi...
4        Drink Distinct All Natural Soda Pineapple Coco...
                               ...                        
49672                                 Organic Hass Avocado
49673                                 Organic Baby Spinach
49674                                 Organic Strawberries
49675                               Bag of Organic Bananas
49676                                               Banana
Name: product_name, Length: 49677, dtype: object

In [72]:
qqq[qqq["product_name"].str.lower().str.contains("butter")]

,product_id,count,product_name,aisle_id,department_id
14,30149,1,Peanut Butter Banana,100,21
38,41967,1,Coconut Cardamom Almond Butter,88,13
106,23741,1,Natural Peanut Butter With Flaxseed,88,13
140,10225,2,Old Fashioned Pumpkin Butter,88,13
160,4650,2,Butter Rum Individually Wrapped Hard Candy,100,21
...,...,...,...,...,...
49490,16953,20033,Creamy Peanut Butter,88,13
49509,30450,21490,Creamy Almond Butter,88,13
49510,7948,21509,Organic Unsalted Butter,36,16
49579,33000,33908,Pure Irish Butter,36,16


In [55]:
qqq[qqq["product_name"].str.lower() == "bread"]

,product_id,count,product_name,aisle_id,department_id
43900,20479,845,Bread,112,3


In [76]:
# bread
cov_itemset(q, set([5077]), absolute=True)
BREAD = 5077

In [77]:
# milk
cov_itemset(q, set([27845]), absolute=True)
MILK = 27845

In [78]:
# banana
cov_itemset(q, set([24852]), absolute=True)
BANANA = 24852

In [79]:
# butter
cov_itemset(q, set([42736]), absolute=True)
BUTTER = 42736

In [75]:
print(cov_itemset(q, set([5077]), absolute=True))
print(cov_itemset(q, set([27845]), absolute=True))
print(cov_itemset(q, set([24852]), absolute=True))
print(cov_itemset(q, set([5077, 27845]), absolute=True))
print(cov_itemset(q, set([5077, 42736]), absolute=True))
print(cov_itemset(q, set([5077, 27845, 24852, 42736]), absolute=True))

60816
137905
472565
3156
720
37


In [ ]:
print(support(q, ({BREAD}, {MILK}), absolute=True))
print(confidence(q, ({BREAD}, {MILK})))
print(coverage(q, ({BREAD}, {MILK})))
print(support(q, ({MILK}, {BREAD}), absolute=True))
print(confidence(q, ({MILK}, {BREAD})))

3156


TypeError: confidence() got an unexpected keyword argument 'absolute'

## 3. Apriori Algorithm

Before we start with the details on how to efficiently search for association rules, let's first
state that the number of possible association rules is exponential in the number of items. We can
choose any item as the consequent and any subset of the items as the antecedent. However, most of
these association rules are not interesting and have no practical value. What we usually want to
find (at least in the most basic cases) are association rules that have large enough support and
confidence.

To find an useful association rule is not much different from finding an interesting/frequent
itemset. It is said that for brick-and-mortar stores, the reasonable threshold could be around 1%
of the transactions. For online stores, where the number of products is much larger, the threshold
is usually even lower ~0.1%.

In practice, we strive to obtain not too many frequent itemsets and association rules. The computed
results are usually presented to or interpreted by a human. Each additional association rule
candidate needs some action to be taken. It is quite normal to modify the thresholds for support
and confidence to modify the number of association rules.

### 3.1 Frequent Itemsets

Frequent itemsets are the sets of items that appear together in a transaction with a frequency
higher than a given threshold. In other words, we are interested in itemsets with "high" (that
depends on the context) coverage.

**Monotonicity Property**: If `A` is a frequent itemset, then all subsets of `A` are frequent
itemsets.

Can you prove this property?

---

Below, the product names are used for simplicity, but you should adapt this to the format you
decided on in the previous step. Compute the frequency of the following itemsets:
- {Bread}
- {Milk}
- {Banana}
- {Bread, Milk}
- {Bread, Butter}
- {Bread, Milk, Butter, Banana}

How would you choose a reasonable frequency threshold to use? How would you approach this problem?
What are the trade-offs?

In [ ]:
# write your code here


### 3.2 Searching for Frequent Itemsets

As a consequence of the above property, if we detect an itemset that is not frequent, then we know
that all its supersets are not frequent either. That observation is the basis of the Apriori
algorithm.

Apriori algorithm is an iterative algorithm for discovering frequent itemsets in a dataset. The
monotonicity property is used to prune the search space. Given a desired support threshold `s`, the
algorithm can be described by the following steps:
- compute the set $L_1$ of all frequent $1$-itemsets
- generate all pairs of distinct $1$-itemsets from $L_1$ and take their union to form the candidate
  set $C_2$ of $2$-itemsets
- filter out the itemsets from $C_2$ if their support is lower than `s`; this results in the set
  $L_2$ of true frequent $2$-itemsets
- we can generalize this process as follows:
    - assume that the set $L_{k-1}$ has been already computed
    - generate all pairs of distinct frequent $(k-1)$-itemsets from $L_{k-1}$ that differ by exactly
      one element; for each such pair, take their union to obtain an itemset `X` of size $k$; if all
      subsets of `X` of size $(k-1)$ are frequent, add `X` to the candidate set $C_k$
    - filter out the itemsets from $C_k$ if their support is lower than `s`; this results in the set
      $L_k$ of true frequent $k$-itemsets
- the process stops when $L_k$ becomes empty or when the maximum itemset length to consider has been
  reached

Implement the Apriori algorithm and apply it to the dataset. Keep in mind that, depending on the
chosen parameters, the number of generated itemsets may become very large. Our goal is to identify
the interesting itemsets without being overwhelmed by an excessive number of them. Therefore, begin
with relatively high support thresholds and gradually lower them to discover additional itemsets.
Adjust thresholds based on the available computational, memory or time resources.

In [ ]:
# write your code here


### 3.3 Association Rules Generation

Suppose we have identified all "interesting" itemsets. We can then generate interesting association
rules (meeting some additional criteria, such as "high" confidence) from them. The procedure to
generate association rules from a frequent itemset `A` can be as simple as follows:
- `A` consists of `n` items,
- we have `n` possible association rules of the form $A \setminus \{a\} \rightarrow \{a\}$ for each
  item $a \in A$,
- we check the confidence of each rule and keep only the ones that meet the confidence threshold.

Prepare a function that for a given itemset `A` and a confidence threshold generates association
rules that meet the confidence threshold.

Generate the set of association rules from the frequent itemsets found in the previous step. Compute
statistics (support, confidence) for them.

In [38]:
# write your code here


### 3.4 Compare the Results

Compare the results you have obtained with the results from any of the available libraries. To
mention a few:
- https://rasbt.github.io/mlxtend/
- https://github.com/tommyod/Efficient-Apriori

Are the results the same or sufficiently close? Are the run times required to run your code
comparable to those of the libraries?

<span style="color:gold">Note:</span> You may want to use some features of the libraries to make it
possible to run the computations on the entire dataset. For example, consider using the
`min_support` (start with higher required support and then lower it), `max_len` (start with smaller
max_len and then increase it), `low_memory` parameters in the `mlxtend` library or transform the
entire dataset into a sparse matrix.